### 정렬, Aggregation 함수, Groupby 적용

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./titanic/train.csv')

#### DataFrame, Series의 정렬 - sort_values()

DataFrame과 Series 정렬을 위해서는 sort_values() 메서드를 이용합니다. sort_values()는 RDBMS의 order by 키워드와 매우 유사합니다.</br>
sort_values()의 주요 입력 파라미터는 by, ascending, inplace입니다.</br>
* by로 특정 칼럼을 입력하면 해당 칼럼으로 정렬을 수행합니다.
* ascending = True(기본값)은, 오름차순, False는 내림차순으로 정렬합니다.
* inplace = False는 기존 DataFrame은 그대로 유지되며 정렬된 DataFrame을 반환합니다. inplace = True는 DataFrame의 정렬 결과를 그대로 적용합니다.

In [3]:
# 'Name'기준으로 오름차순으로 정렬
df_sorted = df.sort_values(by = ['Name'])
df_sorted.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
845,846,0,3,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S
279,280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.25,NaN,S
308,309,0,2,"Abelson, Mr. Samuel",male,30.0,1,0,P/PP 3381,24.00,NaN,C
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.00,NaN,C


In [4]:
# 'Pclass', 'Name'기준으로 내립차순으로 정렬
df_sorted2 = df.sort_values(by= ['Pclass', 'Name'], ascending = False)
df_sorted2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5,NaN,S
153,154,0,3,"van Billiard, Mr. Austin Blyler",male,40.5,0,2,A/5. 851,14.5,NaN,S
282,283,0,3,"de Pelsmaeker, Mr. Alfons",male,16.0,0,0,345778,9.5,NaN,S
286,287,1,3,"de Mulder, Mr. Theodore",male,30.0,0,0,345774,9.5,NaN,S
559,560,1,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,36.0,1,0,345572,17.4,NaN,S


#### Aggregation 함수 적용

 DataFame에서 min(), max(), count()와 같은 aggregation 함수의 적용은 RDBMS SQL의 aggregation 함수 적용과 유사합니다. </br>
 하지만, DataFrame의 경우 DataFrame에서 바로 aggregation을 호출할 경우 모든 칼럼에서 해당 aggregation을 적용합니다.

In [5]:
# 모든 칼럼에 count() 결과 반영
df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [6]:
# 특정 칼럼에 aggregation 함수를 적용하기 위해서는 함수를 추출 후에 함수 적용을 하면 됩니다.
df[['Age', 'Fare']].mean()

Age     29.699118
Fare    32.204208
dtype: float64

#### groupby() 적용

Data의 groupby()는 RDBMS SQL의 groupby 키워드와 유사하면서도 다른 면이 있습니다.</br>
SQL 판다스 모두 groupby를 분석 작업에 매우 많이 활용합니다. 

In [7]:
# 'Pclass'로 그룹화한 object를 count()집계함수(aggregation function)으로 계산합니다.
df_groupby = df.groupby(by = 'Pclass')
print(type(df_groupby), '\n')
df_groupby.count()

<class 'pandas.core.groupby.generic.DataFrameGroupBy'> 



,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [8]:
# 'Pclass'로 groupby한 뒤 'PassengerId'와 'Survived'로 필터링하여 count() 함수를 수행합니다.
df_groupby2 = df.groupby('Pclass')[['PassengerId', 'Survived']].count()
df_groupby2

,PassengerId,Survived
Pclass,,
1,216,216
2,184,184
3,491,491


DataFrame의 groupby()와 SQL의 group by 키워드의 차이는 SQL의 경우 서로 다른 aggregation 함수를 적용할 경우에는 SELECT 절에 나열하면 되지만,</br> DataFrame groupby()의 경우 적용하려는 여러 개의 aggregation 함수명을 DataFrameGroupby 객체의 agg() 내에 인자로 입력해서 사용한다는 점입니다.</br>

In [9]:
# agg()를 사용하여 여러 aggregation 함수를 적용합니다.
df.groupby('Pclass')['Age'].agg([max,min])

,max,min
Pclass,,
1,80.0,0.92
2,70.0,0.67
3,74.0,0.42


In [10]:
# agg() 내에 입력값으로 딕셔너리 형태로 aggregation이 적용될 칼럼과 aggregation 함수를 입력합니다.
agg_format = {'Age':'max',
              'SibSp':'sum',
              'Fare':'mean'}
df.groupby('Pclass').agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,80.0,90,84.154687
2,70.0,74,20.662183
3,74.0,302,13.675550


### 결손 데이터 처리하기

판다스는 결손 데이터를 처리하는 편리한 api를 제공합니다. 결손 데이터는 칼럼에 값이 없는, 즉 null인 경우를 의미하며, 이를 넘파이의 NaN으로 표시합니다.</br>
머신러닝 알고리즘은 이 NaN 값을 처리하지 않으므로 이를 다른 데이터로 대체해야합니다. </br>
NaN 값은 평균, 총합, 등의 함수 연산 시 제외 됩니다.</br>
예를 들어 데이터 100개 중 10개가 NaN이라면, 이 100개의 데이터의 평균은 NaN가 아닌 90개의 데이터에 대해서만 연산을 합니다.</br>
NaN 여부를 확인하는 API는 isna()이며, NaN 값을 다른 값으로 대체하는 API는 fillna()입니다.

#### isna()로 결손 데이터 여부 확인

In [11]:
# DataFrame에 isna()를 수행하면 모든 칼럼 값이 NaN인지 아닌지 True, False로 알려줍니다.(True이면 결손 데이터)
df.isna().head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False


In [12]:
# 결손 데이터의 개수는 isna() 결과에 sum() 함수를 추가해 구할 수 있습니다.
# sum()을 호출 시 True는 내부적으로 숫자 1로, False는 숫자 0으로 변환됩니다.
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### fillna()로 결손 데이터 대체하기

fillna()를 이용하면 결손 데이터를 편리하게 다른 값으로 대체할 수 있습니다.

In [13]:
# DataFrame의 'Cabin'칼럼의 NaN값을 'C000'으로 대체합니다.
# inplace를 통해 본래 DataFrame을 변경할 수 있습니다.
df['Cabin'] = df['Cabin'].fillna('C000')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C000,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C000,S


In [14]:
# 'Age'칼럼의 NaN 값을 평균나이로, 'Embarked' 칼럼의 NaN값을 'S'로 대체해 모든 결손 데이터를 처리합니다.
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna('S')
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64